In [16]:
import numpy as np
import pandas as pd

def read_input_file(file_path):
    data = pd.read_csv(file_path, index_col=0)
    supply = data.iloc[:-1, -1].values
    demand = data.iloc[-1, :-1].values
    cost = data.iloc[:-1, :-1].values
    return supply, demand, cost

def northwest_corner_rule(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    rows, cols = len(supply), len(demand)
    solution = np.zeros((rows, cols))

    i, j = 0, 0
    while i < rows and j < cols:
        allocation = min(supply[i], demand[j])
        solution[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation

        if supply[i] == 0:
            i += 1
        elif demand[j] == 0:
            j += 1

    return solution

def minimum_cost_method(supply, demand, cost):
    supply = supply.copy()
    demand = demand.copy()
    cost = cost.astype(float)  # Convert cost matrix to float
    rows, cols = len(supply), len(demand)
    solution = np.zeros((rows, cols))

    while np.any(supply > 0) and np.any(demand > 0):
        min_cost_index = np.unravel_index(np.argmin(cost, axis=None), cost.shape)
        i, j = min_cost_index
        allocation = min(supply[i], demand[j])
        solution[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation
        cost[i, j] = np.inf  # Mark as used

    return solution

def minimum_row_cost_method(supply, demand, cost):
    supply = supply.copy()
    demand = demand.copy()
    cost = cost.astype(float)  # Ensure cost is float to handle np.inf
    rows, cols = len(supply), len(demand)
    solution = np.zeros((rows, cols))

    while np.any(supply > 0) and np.any(demand > 0):
        for i in range(rows):
            # Find the column with the minimum cost in the current row
            min_cost_index = np.argmin(cost[i, :])
            j = min_cost_index

            # Determine the allocation based on the minimum cost
            allocation = min(supply[i], demand[j])
            solution[i, j] = allocation
            supply[i] -= allocation
            demand[j] -= allocation

            # After allocation, if a supply or demand is exhausted, set cost to infinity to avoid future selection
            if supply[i] == 0:
                cost[i, :] = np.inf  # Mark entire row as "used"
            if demand[j] == 0:
                cost[:, j] = np.inf  # Mark entire column as "used"

    return solution



def vogels_method(supply, demand, cost):
    supply = supply.copy()
    demand = demand.copy()
    cost = cost.astype(float)  # Ensure cost is float to handle np.inf
    rows, cols = len(supply), len(demand)
    solution = np.zeros((rows, cols))

    while np.any(supply > 0) and np.any(demand > 0):
        # Pre-compute penalties
        penalties = []

        for i in range(rows):
            row = cost[i, :]
            valid_costs = row[row != np.inf]
            if len(valid_costs) > 1:
                penalties.append((valid_costs[0] - valid_costs[1], 'row', i))
            elif len(valid_costs) == 1:
                penalties.append((valid_costs[0], 'row', i))

        for j in range(cols):
            col = cost[:, j]
            valid_costs = col[col != np.inf]
            if len(valid_costs) > 1:
                penalties.append((valid_costs[0] - valid_costs[1], 'col', j))
            elif len(valid_costs) == 1:
                penalties.append((valid_costs[0], 'col', j))

        # Find the maximum penalty
        max_penalty = max(penalties, key=lambda x: x[0])

        # Determine the allocation based on the max penalty
        if max_penalty[1] == 'row':
            i = max_penalty[2]
            j = np.argmin(cost[i, :])
        else:
            j = max_penalty[2]
            i = np.argmin(cost[:, j])

        # Perform allocation
        allocation = min(supply[i], demand[j])
        solution[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation

        # Mark cell as used
        cost[i, j] = np.inf

    return solution

def transportation_simplex(supply, demand, cost, initial_solution):
    supply = supply.copy()
    demand = demand.copy()
    cost = cost.copy()
    solution = initial_solution.copy()
    rows, cols = solution.shape

    # Initialize dual variables u and v
    u = np.full(rows, np.nan)  # Initialize u with NaN (undefined values)
    v = np.full(cols, np.nan)  # Initialize v with NaN
    u[0] = 0  # Arbitrary starting point for u[0]

    # Efficient calculation of u and v values
    changed = True
    while changed:
        changed = False
        for i in range(rows):
            for j in range(cols):
                if solution[i, j] > 0:  # Only consider non-zero allocations
                    if np.isnan(u[i]) and not np.isnan(v[j]):
                        u[i] = cost[i, j] - v[j]
                        changed = True
                    elif np.isnan(v[j]) and not np.isnan(u[i]):
                        v[j] = cost[i, j] - u[i]
                        changed = True

    # Calculate improvement indices only for unallocated cells (0 cells)
    improvement_indices = np.full((rows, cols), np.nan)
    for i in range(rows):
        for j in range(cols):
            if solution[i, j] == 0:  # Only calculate for empty cells
                improvement_indices[i, j] = cost[i, j] - (u[i] + v[j])

    # Check for optimality: If all indices are non-negative, the solution is optimal
    if np.all(improvement_indices >= 0):
        return solution

    # Find the entering variable (most negative improvement index)
    i, j = np.unravel_index(np.argmin(improvement_indices), improvement_indices.shape)

    # Loop finding and adjusting the flow (part still to implement)
    # Adjust the solution (this part remains to be optimized further)
    
    # Here you could add your cycle detection and flow adjustment mechanism

    return solution


def main():
    # Load data
    file_path = "\\transportation_data.csv"
    supply, demand, cost = read_input_file(file_path)
    cost = cost.astype(float)  # Convert cost matrix to float

    # Northwest Corner Rule
    nwc_solution = northwest_corner_rule(supply, demand)
    print("Northwest Corner Rule Solution:")
    print(nwc_solution)

    # Minimum Cost Method
    min_cost_solution = minimum_cost_method(supply, demand, cost)
    print("Minimum Cost Method Solution:")
    print(min_cost_solution)
    
    # Minimum Row Cost Method
    row_cost_solution = minimum_row_cost_method(supply, demand, cost)
    print("Minimum Row Cost Method Solution:")
    print(row_cost_solution)

    # Vogel's Method
    vogels_solution = vogels_method(supply, demand, cost)
    print("Vogel's Method Solution:")
    print(vogels_solution)

    # Transportation Simplex (using Vogel's as initial solution)
    optimal_solution = transportation_simplex(supply, demand, cost, vogels_solution)
    print("Optimal Solution (Transportation Simplex):")
    print(optimal_solution)

if __name__ == "__main__":
    main()


Northwest Corner Rule Solution:
[[48.  0.  0.  0.]
 [ 0. 28. 10.  0.]
 [ 0.  0. 22.  2.]
 [ 0.  0.  0. 17.]
 [ 0.  0.  0.  1.]]
Minimum Cost Method Solution:
[[16.  0. 32.  0.]
 [ 8.  0.  0.  1.]
 [24.  0.  0.  0.]
 [ 0. 17.  0.  0.]
 [ 0. 11.  0. 19.]]
Minimum Row Cost Method Solution:
[[ 7.  0. 32.  0.]
 [ 0. 28.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. 20.]]
Vogel's Method Solution:
[[ 0.  0. 32.  0.]
 [38.  0.  0.  0.]
 [ 4.  0.  0. 20.]
 [ 6.  0.  0.  0.]
 [ 0. 28.  0.  0.]]
Optimal Solution (Transportation Simplex):
[[ 0.  0. 32.  0.]
 [38.  0.  0.  0.]
 [ 4.  0.  0. 20.]
 [ 6.  0.  0.  0.]
 [ 0. 28.  0.  0.]]
